<header style="padding:1px;background:#f9f9f9;border-top:3px solid #00b2b1"><img id="Teradata-logo" src="https://www.teradata.com/Teradata/Images/Rebrand/Teradata_logo-two_color.png" alt="Teradata" width="220" align="right" />

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>ModelOps demo - Python XGBoost using Git</b>
</header>

![image](images/git_meth.png) 

## Introduction

This notebook will cover the Operationalization of the PIMA diabetes use case with Python XGBoost model format. **XGBoost** is an optimized distributed gradient boosting library designed to be highly efficient, flexible and portable. It is one of the most used libraries by the community that solve many data science problems in a fast and accurate way.

In this example, we will use the XGBoost algorithm to generate both **Python Joblib and PMML** model formats and operationalize them through ModelOps in the same Model Catalog than other trained models based on other libraries.

## Steps in this Notebook

<li>1. Configure the Environment </li>
    <li>2. Connect to Vantage</li>
    <li>3. Define Training function </li>
    <li>4. Define Evaluate function </li>
    <li>5. Define Scoring function</li>
    <li>6. Define Model Metadata</li>
    <li>7. Commit and Push to Git to let ModelOps manage</li>
    <li>8. ModelOps full lifecycle till deployment</li>
    <li>9. ModelOps Monitoring</li>


## Step 1. Configure the Environment

Here, we import the required libraries, set environment variables and environment paths (if required).

### 1.1 Libraries installation

A restart of the Kernel is needed to confirm changes


In [ ]:
%pip install -q teradataml==17.20.0.3 aoa==7.0.1 pandas==1.1.5 xgboost==0.90 nyoka==4.3.0 scikit-learn==0.24.2 matplotlib==3.5.2

### 1.2 Libraries import

In [ ]:
from teradataml import (
    create_context, 
    remove_context,
    get_context,
    get_connection,
    DataFrame,
    configure
)
import os
import getpass
import logging
import sys


## Step 2. Connect to Vantage

<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, then use down arrow to go to next cell. Begin running steps with Shift + Enter keys.</p>

In [ ]:
%run -i ../UseCases/startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)
eng.execute('''SET query_band='DEMO=09_ModelOps_GIT_PIMA_Python_XGboost.ipynb;' UPDATE FOR SESSION; ''')

# configure byom/val installation
configure.val_install_location = "VAL"
configure.byom_install_location = "MLDB"

# set the path to the local project repository for this model demo
model_local_path = '~/modelops-demo-models/model_definitions/pima_python_xgboost'
res = os.system(f'mkdir -p {model_local_path}/model_modules')

## Step 3. Define Training Function

The training function takes the following shape

```python
def train(context: ModelContext, **kwargs):
    aoa_create_context()
    
    # your training code
    
    # save your model
    joblib.dump(model, f"{context.artifact_output_path}/model.joblib")
    
    record_training_stats(...)
```

You can execute this from the CLI or directly within the notebook as shown.

In [ ]:
%%writefile $model_local_path/model_modules/training.py
from xgboost import XGBClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from nyoka import xgboost_to_pmml
from teradataml import DataFrame
from aoa import (
    record_training_stats,
    save_plot,
    aoa_create_context,
    ModelContext
)

import joblib


def train(context: ModelContext, **kwargs):
    aoa_create_context()

    feature_names = context.dataset_info.feature_names
    target_name = context.dataset_info.target_names[0]

    # read training dataset from Teradata and convert to pandas
    train_df = DataFrame.from_query(context.dataset_info.sql)
    train_pdf = train_df.to_pandas(all_rows=True)

    # split data into X and y
    X_train = train_pdf[feature_names]
    y_train = train_pdf[target_name]

    print("Starting training...")

    # fit model to training data
    model = Pipeline([('scaler', MinMaxScaler()),
                      ('xgb', XGBClassifier(eta=context.hyperparams["eta"],
                                            max_depth=context.hyperparams["max_depth"]))])

    model.fit(X_train, y_train)

    print("Finished training")

    # export model artefacts
    joblib.dump(model, f"{context.artifact_output_path}/model.joblib")

    # we can also save as pmml so it can be used for In-Vantage scoring etc.
    xgboost_to_pmml(pipeline=model, col_names=feature_names, target_name=target_name,
                    pmml_f_name=f"{context.artifact_output_path}/model.pmml")

    print("Saved trained model")

    from xgboost import plot_importance
    model["xgb"].get_booster().feature_names = feature_names
    plot_importance(model["xgb"].get_booster(), max_num_features=10)
    save_plot("feature_importance.png", context=context)

    feature_importance = model["xgb"].get_booster().get_score(importance_type="weight")

    print("Recording training stats")

    record_training_stats(train_df,
                          features=feature_names,
                          targets=[target_name],
                          categorical=[target_name],
                          feature_importance=feature_importance,
                          context=context)
    
    print("All done!")

In [ ]:
# Define the ModelContext to test with. The ModelContext is created and managed automatically by ModelOps 
# when it executes your code via CLI / UI. However, for testing in the notebook, you can define as follows

# define the training dataset 
sql = """
SELECT 
    F.*, D.hasdiabetes
FROM PIMA_PATIENT_FEATURES F 
JOIN PIMA_PATIENT_DIAGNOSES D
ON F.patientid = D.patientid
    WHERE D.patientid MOD 5 <> 0
"""

feature_metadata =  {
    "database": "demo_user",
    "table": "aoa_statistics_metadata"
}
hyperparams = {"max_depth": 5, "eta": 0.2}

entity_key = "PatientId"
target_names = ["HasDiabetes"]
feature_names = ["NumTimesPrg", "PlGlcConc", "BloodP", "SkinThick", "TwoHourSerIns", "BMI", "DiPedFunc", "Age"]
 
from aoa import ModelContext, DatasetInfo

dataset_info = DatasetInfo(sql=sql,
                           entity_key=entity_key,
                           feature_names=feature_names,
                           target_names=target_names,
                           feature_metadata=feature_metadata)

ctx = ModelContext(hyperparams=hyperparams,
                   dataset_info=dataset_info,
                   artifact_output_path="./artifacts",
                   model_version="v1",
                   model_table="aoa_model_v1")

sys.path.append(os.path.expanduser(f"{model_local_path}/model_modules"))
import training
training.train(context=ctx)

In [ ]:
# Check the generated files
!ls -lh artifacts

## Step 4. Define Evaluation Function

The evaluation function takes the following shape

```python
def evaluate(context: ModelContext, **kwargs):
    aoa_create_context()

    # read your model
    model = joblib.load(f"{context.artifact_input_path}/model.joblib")
    
    # your evaluation logic
    
    record_evaluation_stats(...)
```

You can execute this from the CLI or directly within the notebook as shown.

In [ ]:
%%writefile $model_local_path/model_modules/evaluation.py
from sklearn import metrics
from teradataml import DataFrame, copy_to_sql
from aoa import (
    record_evaluation_stats,
    save_plot,
    aoa_create_context,
    ModelContext
)

import joblib
import json
import numpy as np
import pandas as pd


def evaluate(context: ModelContext, **kwargs):

    aoa_create_context()

    model = joblib.load(f"{context.artifact_input_path}/model.joblib")

    feature_names = context.dataset_info.feature_names
    target_name = context.dataset_info.target_names[0]

    test_df = DataFrame.from_query(context.dataset_info.sql)
    test_pdf = test_df.to_pandas(all_rows=True)

    X_test = test_pdf[feature_names]
    y_test = test_pdf[target_name]

    print("Scoring")
    y_pred = model.predict(X_test)

    y_pred_tdf = pd.DataFrame(y_pred, columns=[target_name])
    y_pred_tdf["PatientId"] = test_pdf["PatientId"].values

    evaluation = {
        'Accuracy': '{:.2f}'.format(metrics.accuracy_score(y_test, y_pred)),
        'Recall': '{:.2f}'.format(metrics.recall_score(y_test, y_pred)),
        'Precision': '{:.2f}'.format(metrics.precision_score(y_test, y_pred)),
        'f1-score': '{:.2f}'.format(metrics.f1_score(y_test, y_pred))
    }

    with open(f"{context.artifact_output_path}/metrics.json", "w+") as f:
        json.dump(evaluation, f)

    metrics.plot_confusion_matrix(model, X_test, y_test)
    save_plot('Confusion Matrix', context=context)

    metrics.plot_roc_curve(model, X_test, y_test)
    save_plot('ROC Curve', context=context)

    from xgboost import plot_importance
    model["xgb"].get_booster().feature_names = feature_names
    plot_importance(model["xgb"].get_booster(), max_num_features=10)
    save_plot("feature_importance.png", context=context)

    feature_importance = model["xgb"].get_booster().get_score(importance_type="weight")

    predictions_table = "predictions_tmp"
    copy_to_sql(df=y_pred_tdf, table_name=predictions_table, index=False, if_exists="replace", temporary=True)

    record_evaluation_stats(features_df=test_df,
                            predicted_df=DataFrame.from_query(f"SELECT * FROM {predictions_table}"),
                            feature_importance=feature_importance,
                            context=context)

    print("All done!")

In [ ]:
# Define the ModelContext to test with. The ModelContext is created and managed automatically by ModelOps 
# when it executes your code via CLI / UI. However, for testing in the notebook, you can define as follows

# define the evaluation dataset 
sql = """
SELECT 
    F.*, D.hasdiabetes 
FROM PIMA_PATIENT_FEATURES F 
JOIN PIMA_PATIENT_DIAGNOSES D
ON F.patientid = D.patientid
    WHERE D.patientid MOD 5 = 0
"""

dataset_info = DatasetInfo(sql=sql,
                           entity_key=entity_key,
                           feature_names=feature_names,
                           target_names=target_names,
                           feature_metadata=feature_metadata)

ctx = ModelContext(hyperparams=hyperparams,
                   dataset_info=dataset_info,
                   artifact_output_path="./artifacts",
                   artifact_input_path="./artifacts",
                   model_version="v1",
                   model_table="aoa_model_v1")

import evaluation
evaluation.evaluate(context=ctx)

# view evaluation results
import json
with open(f"{ctx.artifact_output_path}/metrics.json") as f:
    print(json.load(f))

In [ ]:
# Check the generated files
!ls -lh artifacts

## Step 5. Define Scoring Function

The scoring function takes the following shape

```python
def score(context: ModelContext, **kwargs):
    aoa_create_context()

    # read your model
    model = joblib.load(f"{context.artifact_input_path}/model.joblib")
    
    # your evaluation logic
    
    record_scoring_stats(...)
```

You can execute this from the CLI or directly within the notebook as shown.

In [ ]:
%%writefile $model_local_path/model_modules/scoring.py
from teradataml import copy_to_sql, DataFrame
from aoa import (
    record_scoring_stats,
    aoa_create_context,
    ModelContext
)

import joblib
import pandas as pd


def score(context: ModelContext, **kwargs):

    aoa_create_context()

    model = joblib.load(f"{context.artifact_input_path}/model.joblib")

    feature_names = context.dataset_info.feature_names
    target_name = context.dataset_info.target_names[0]
    entity_key = context.dataset_info.entity_key

    features_tdf = DataFrame.from_query(context.dataset_info.sql)
    features_pdf = features_tdf.to_pandas(all_rows=True)

    print("Scoring")
    predictions_pdf = model.predict(features_pdf[feature_names])

    print("Finished Scoring")

    # store the predictions
    predictions_pdf = pd.DataFrame(predictions_pdf, columns=[target_name])
    predictions_pdf[entity_key] = features_pdf.index.values
    # add job_id column so we know which execution this is from if appended to predictions table
    predictions_pdf["job_id"] = context.job_id

    # teradataml doesn't match column names on append.. and so to match / use same table schema as for byom predict
    # example (see README.md), we must add empty json_report column and change column order manually (v17.0.0.4)
    # CREATE MULTISET TABLE pima_patient_predictions
    # (
    #     job_id VARCHAR(255), -- comes from airflow on job execution
    #     PatientId BIGINT,    -- entity key as it is in the source data
    #     HasDiabetes BIGINT,   -- if model automatically extracts target
    #     json_report CLOB(1048544000) CHARACTER SET UNICODE  -- output of
    # )
    # PRIMARY INDEX ( job_id );
    predictions_pdf["json_report"] = ""
    predictions_pdf = predictions_pdf[["job_id", entity_key, target_name, "json_report"]]

    copy_to_sql(df=predictions_pdf,
                schema_name=context.dataset_info.predictions_database,
                table_name=context.dataset_info.predictions_table,
                index=False,
                if_exists="append")
    
    print("Saved predictions in Teradata")

    # calculate stats
    predictions_df = DataFrame.from_query(f"""
        SELECT 
            * 
        FROM {context.dataset_info.get_predictions_metadata_fqtn()} 
            WHERE job_id = '{context.job_id}'
    """)

    record_scoring_stats(features_df=features_tdf,
                         predicted_df=predictions_df,
                         context=context)

    print("All done!")

In [ ]:
# Define the ModelContext to test with. The ModelContext is created and managed automatically by ModelOps 
# when it executes your code via CLI / UI. However, for testing in the notebook, you can define as follows

# define the scoring dataset 

sql = """
SELECT 
    F.*
FROM PIMA_PATIENT_FEATURES F 
    WHERE F.patientid MOD 5 = 0
"""

# where to store predictions
predictions = {
    "database": "demo_user",
    "table": "pima_patient_predictions_tmp"
}

import uuid
job_id=str(uuid.uuid4())

dataset_info = DatasetInfo(sql=sql,
                           entity_key=entity_key,
                           feature_names=feature_names,
                           target_names=target_names,
                           feature_metadata=feature_metadata,
                           predictions=predictions)

ctx = ModelContext(hyperparams=hyperparams,
                   dataset_info=dataset_info,
                   artifact_output_path="./artifacts",
                   artifact_input_path="./artifacts",
                   model_version="v1",
                   model_table="aoa_model_v1",
                   job_id=job_id)

import scoring
scoring.score(context=ctx)

In [ ]:
DataFrame.from_query(f"SELECT * FROM pima_patient_predictions_tmp WHERE job_id='{job_id}'")

In [ ]:
# Clean up

os.system('rm -f artifacts/*')

try:
    get_context().execute(f"DROP TABLE aoa_model_v1")
except: 
    pass

try:
    get_context().execute(f"DROP TABLE pima_patient_predictions_tmp")
except: 
    pass

## Step 6. Define Model Metadata

Now let's create the configuration files.

Requirements file with the dependencies and versions:

In [ ]:
%%writefile $model_local_path/model_modules/requirements.txt
xgboost==0.90
scikit-learn==0.24.2
teradataml==17.20.0.3
nyoka==4.3.0
aoa==7.0.1
matplotlib==3.5.2

The hyper parameter configuration (default values):

In [ ]:
%%writefile $model_local_path/config.json
{
   "hyperParameters": {
      "eta": 0.2,
      "max_depth": 6
   }
}

The model configuration:

In [ ]:
%%writefile $model_local_path/model.json
{
    "id": "5b41d4d4-7236-54ab-846a-01c3151e1fd9",
    "name": "Python PIMA Quickstart",
    "description": "Python PIMA Quickstart for Diabetes Prediction",
    "language": "python"
}

## Step 7.Commit and push changes into GIT to manage from ModelOps

Run the command below to commit and push changes to our forked repository, so ModelOps can fetch the changes to the model.

In [ ]:
!cd $model_local_path/../.. && git add . && git commit -m "Added PIMA Quickstart demo model ⚡" && git push

Now that changes are pushed, you can make the lifecycle inside **ModelOps User Interface**, plan for new trainings, evaluations, scorings. Compare models and operationalize into Production with automated Monitoring and alerting capabilities.

## Step 8. ModelOps full lifecycle till deployment


Use or Create a Project with the git code repository with the model code, then you should see the model in the catalog already created

<img src="images/08_01.png" alt="Model Catalog with inDB"/>

Select the Model and then click Train a new Model. Use default hyper-parameters. This will launch the training job with the training script we generated and pushed to Git.

<img src="images/08_02.png" alt="Train"/>

<img src="images/08_03.png" alt="Train job" width="500" height="500"/>

<img src="images/08_04.png" alt="Train finished" width="500" height="500"/>

When Model is trained a new Model Id is created and you can get inside the Model Lifecycle screen to review artifacts and other details

<img src="images/08_06.png" alt="Model lifecycle"/>

Now, let's evaluate the Model, click the button and select the evaluation dataset. This will launch the evaluation job with the training script we generated and pushed to Git.

<img src="images/08_07.png" alt="Evaluation" width="500" height="500"/> <img src="images/08_08.png" alt="Evaluation job" width="500" height="500"/>



When evaluation job is finished a Model evaluation Report is generated with the metrics and charts that evaluation script generates

<img src="images/08_26.png" alt="Model Report" />

Now, let's approve the model and provide an approval description

<img src="images/08_09.png" alt="Approval" />

<img src="images/08_10.png" alt="Approval description" width="500" height="500"/>

The model is ready to be deployed. Let's deploy using a Batch scheduling option - Run it manual

<img src="images/08_11.png" alt="Deployment Engine" width="500" height="500"/>

<img src="images/08_12.png" alt="Deployment Publish" width="500" height="500"/>

<img src="images/08_13.png" alt="Deployment Schedule" width="500" height="500"/>


Go and try this Step by yourself. Launch ModelOps from this button below:

[![image](images/launchModelOps.png)](/modelops)

## Step 9. ModelOps Monitoring

Now the model is deployed and a new Deployment appears in the deployment screen


<img src="images/08_15.png" alt="Deploymet" />


You can run jobs manually from here, review history of executions and view the predictions for a specific job

<img src="images/08_16.png" alt="Deployment Run" width="500" height="500"/>

<img src="images/08_17.png" alt="Deployment Jobs" />

<img src="images/08_18.png" alt="Deployment view" width="500" height="500" />

<img src="images/08_19.png" alt="Deployment predictions" width="500" height="500"/>

<img src="images/08_20.png" alt="Deployment" width="500" height="500"/>


From the Feature Drift and Prediction Drift tabs you can check on the monitoring of the data drift

<img src="images/08_22.png" alt="Feature Drift" />

<img src="images/08_21.png" alt="Prediction Drift" />

<img src="images/08_23.png" alt="Performance Monitoring" />




From the Performance Drift, you can review multiple evaluations, let's evaluate the model with a new dataset. We create a new evaluation dataset with this query:
    
    SELECT * FROM pima_patient_diagnoses F WHERE F.patientid MOD 8 <> 0
    
<img src="images/08_24.png" alt="Evaluate" width="500" height="500" />

and now see the evolution of the metrics

<img src="images/08_25.png" alt="Metrics monitoring" />


With ModelOps you can close the cycle and review make decisions when you need to replace yor model in production, For example, You could get alerting from Data Drift of Performance Drift and you can create multiple versions and compare them, select a champion and deploy new versions that replace existing in Production.

Go and try this Step by yourself. Launch ModelOps from this button below:

[![image](images/launchModelOps.png)](/modelops)

<footer style="padding:10px;background:#f9f9f9;border-bottom:3px solid #394851">©2023 Teradata. All Rights Reserved</footer>